## **Mathematical Notations and Capabilities in `param()`**
> working document exemplifying the evaluation/substitution rules enforced in Pizza3
> 
> ⚠️ some issues have been identified, they are fixed in the meantime

The `param()` class extends the `struct` class and allows **dynamic evaluation** of expressions, **implicit calculations**, and **NumPy-style operations**. 

*Note: Matlab inputs are also enabled as shorthands.*

### Manage dependencies
It is mandatory to import NumPy as np (internal convention) if NumPy arrays are defined in strings

In [30]:
# assuming that cwd is Pizza3/ main folder
# check it with :
'''
# check it with
import os
current_dir = os.getcwd()
print(current_dir)
'''
# import NumPy
import numpy as np
# import param from mstruct module
from pizza.private.mstruct import param
# small function to display the value and the type of type after evaluation
prettyprint = lambda var,value: print(f"{var} = {value} (type: {type(value).__name__})")

### **1. General Expression Substitution**
- Mathematical expressions are stored in strings `""` involving numbers, variables, and operators/functions
- Expressions inside **`${}`** are evaluated dynamically (beyond the simple substitution of ${var} by its content).
- Evaluations support **scalars, lists, NumPy arrays**, and **expressions**.
- Expressions can be extended outside `${}` and involve the combination of several `{}` contents
- Indexing 

#### **Example**


In [68]:
p = param()                    # initialize param. Note that can use also `p = param(a=..., b=...)`
p.a  = [1.0, 0.2, 0.03, 0.004] # Python list
p.b  = np.array([p.a])         # Converts a as row vector
p.c = "10"                     # A number can be stored in a string
p.d = "${a[1]}+${c}"           # Retrieves `a[1]` = 0.2 and add 10 `0.2 + 10`
p.e = "${b[0,1]} + ${a[0]}"    # Evaluates as `0.2 + 1.0`
p.f = "${a}[1]+${c}"           # This notation also works (equivalent to c for the part `${a}[1]`) `0.2 + 10`
p.g = "${b}[0,1] + ${a}[0]"    # However, it should be avoided with NumPy arrays (see below)

✅ **Supported Operations**

- Indexing **lists and arrays**.
- Using **mathematical operators** like `+`, `-`, `*`, `/`.

In [70]:
# à la Matlab practice, type the variable to see its content
p
# alternatively use repr(p)

  -------------:----------------------------------------
              a: [1.0, 0.2, 0.03, 0.004]
               = [1.0, 0.2, 0.03, 0.004]
              b: [1 0.2 0.03 0.004] (double)
              c: 10
               = 10
              d: ${a[1]}+${c}
               = 10.2
              e: ${b[0,1]} + ${a[0]}
               = 1.2
              f: ${a}[1]+${c}
               = 10.2
              g: ${b}[0,1] + ${a}[0]
               = [[1.0, 0.2, 0.03, 0. [...] 0.2, 0.03, 0.004][0]
  -------------:----------------------------------------


parameter list (param object) with 7 definitions

**Global evaluation**

    - Principles:
        a) the variables are evaluated in sequence following their definition order.
        b) an error message is shown if a variable is undefined.
        c) use `paramauto()` if the order of execution needs to be guessed
        d) the depth of the evaluation is also determined by the context
        e) use escape sequences `\${var}` if some expressions should not be evaluated
        f) add `$` at the beginning of an expression to prevent its evaluation/substitution

In [100]:
p.i = "\${a}+1"        # escape `${a}` to prevent its substitution
p.j = ["a","b"]
s = p() # equivalent to s = p.eval()
s


  -------------:----------------------------------------
              a: [1.0, 0.2, 0.03, 0.004]
              b: [1 0.2 0.03 0.004] (double)
              c: 10
              d: 10.2
              e: 1.2
              f: 10.2
              g: [[1.0, 0.2, 0.03, 0. [...] 0.2, 0.03, 0.004][0]
              i: ${a}
              j: [[1.0, 0.2, 0.03, 0. [...]   , 0.03 , 0.004]])]
  -------------:----------------------------------------


structure (struct object) with 9 fields

    - interpretation

In [76]:
# evaluate all expressions and store them in s
s = p() # equivalent to s = p.eval() 
# c and e are equivalent
print("Inner and outer indexing are similar for list")
repr(p("d","f")) # show the evaluation of c and e
print('All values are numeric (float)')
prettyprint("d",s.d)
prettyprint("f",s.f)
# d and f are not equivalent
print("\n","Inner and outer indexing are not similar for NumPy arrays")
repr(p("e","g")) # show the evaluation of d and f
print("only the first value (d) is numeric")
prettyprint("e",s.e)
prettyprint("g",s.g)
print('avoid using outer indexing and keep it within "{}"')

Inner and outer indexing are similar for list
  -------------:----------------------------------------
              d: 10.2
              f: 10.2
  -------------:----------------------------------------
All values are numeric (float)
d = 10.2 (type: float)
f = 10.2 (type: float)

 Inner and outer indexing are not similar for NumPy arrays
  -------------:----------------------------------------
              e: 1.2
              g: [[1.0, 0.2, 0.03, 0. [...] 0.2, 0.03, 0.004][0]
  -------------:----------------------------------------
only the first value (d) is numeric
e = 1.2 (type: float)
g = [[1.0, 0.2, 0.03, 0.004]][0,1] + [1.0, 0.2, 0.03, 0.004][0] (type: str)
avoid using outer indexing and keep it within "{}"


**Special case of nested evaluations**

Expressions can be included in lists. Each list can mix numbers, text, and expressions.
The context will be used to 

In [15]:
p = param()
p.a = [0,1,2]                   # this list is numeric and is already in Python
p.b = '[1,2,"test","${a[1]}"]'  # this list combines param expressions
p.c = '![1,2,"test","${a[1]}"]' # the `!` to force numerical evaluation of expressions in lists
p.d = "${b[3]}*10"              # the expressions can be combined together
p.e = "${c[3]}*10"              # the expressions can be combined together
p.f = [1,2,"test","${a[1]}"]    # evaluations are also applied in native Python lists
s = p.eval()
print(f'b[3] result is a string "{s.b[3]}"')
print(f'c[3] result is a number {s.c[3]}')
print(f'"b[3]*10" result is a number {s.d}')
print(f'"c[3]*10" result is a number {s.e}')
p

b[3] result is a string "1"
c[3] result is a number 1
"b[3]*10" result is a number 10
"c[3]*10" result is a number 10
  -------------:----------------------------------------
              a: [0, 1, 2]
               = [0, 1, 2]
              b: [1,2,"test","${a[1]}"]
               = [1, 2, 'test', '1']
              c: ![1,2,"test","${a[1]}"]
               = [1, 2, 'test', 1]
              d: ${b[3]}*10
               = 10
              e: ${c[3]}*10
               = 10
              f: [1, 2, 'test', '${a[1]}']
               = [1, 2, 'test', 1]
  -------------:----------------------------------------


parameter list (param object) with 6 definitions

In [17]:
# create p and assign some values
p = param()
p.a = "$[1:3]"           # Becomes `[1, 2, 3]`
p.b = "$[1;2;3]"         # Becomes `[[1];[2];[3]]`
p.c = "$[0.1:0.1:0.9]"   # `[0.1, 0.2, 0.3, ..., 0.9]`
p.d = "$[1 2 3; 4 5 6]"  # `[[1,2,3], [4,5,6]]`


In [21]:
# à la Matlab practice, type the variable to see its content
# note the notation [1 2 3]T where T means transpose
# display matches NumPy outputs
p

  -------------:----------------------------------------
              a: $[1:3]
               = [1 2 3] (int64)
              b: $[1;2;3]
               = [1 2 3]T (int64)
              c: $[0.1:0.1:0.9]
               = [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9] (double)
              d: $[1 2 3; 4 5 6]
               = [2×3 int64]
  -------------:----------------------------------------


parameter list (param object) with 4 definitions